In [5]:
## NOTE: This is Python 3 code.
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt # NOTE: This was tested with matplotlib v. 2.1.0
 
from sklearn.preprocessing import StandardScaler

In [3]:
#########################
#
# Data Generation Code
#
#########################
df = pd.read_csv('../_data/operations_imputed_CLEAN.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91862 entries, 0 to 91861
Data columns (total 63 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         91862 non-null  int64  
 1   op_id              91862 non-null  int64  
 2   subject_id         91862 non-null  int64  
 3   hadm_id            91862 non-null  int64  
 4   opdate             91862 non-null  int64  
 5   age                91862 non-null  int64  
 6   sex                91862 non-null  object 
 7   weight             90910 non-null  float64
 8   height             91326 non-null  float64
 9   race               91862 non-null  object 
 10  asa                89532 non-null  float64
 11  emop               91862 non-null  int64  
 12  department         91862 non-null  object 
 13  antype             91862 non-null  object 
 14  icd10_pcs          91862 non-null  object 
 15  category_desc      91862 non-null  object 
 16  desc_short         918

In [8]:

#########################
#
# Perform PCA on the data
#
#########################
# First center and scale the data

cols_to_drop = ['op_id','subject_id','hadm_id','opdate','LOS','is_outlier','prolonged_LOS']

scaler = StandardScaler()
X = df.drop(cols_to_drop,axis=1)
y = df['LOS']
X = scaler.fit_transform(X)
dfx = pd.DataFrame(data=X,columns=df_final.columns[:-2])
dfx.head(10)



ValueError: could not convert string to float: 'F'

In [ ]:

scaled_data = preprocessing.scale(df.T)
 
pca = PCA() # create a PCA object
pca.fit(scaled_data) # do the math
pca_data = pca.transform(scaled_data) # get PCA coordinates for scaled_data
 
#########################
#
# Draw a scree plot and a PCA plot
#
#########################
 
#The following code constructs the Scree plot
per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
 
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot')
plt.show()
 
#the following code makes a fancy looking plot using PC1 and PC2
pca_df = pd.DataFrame(pca_data, index=[*wt, *ko], columns=labels)
 
plt.scatter(pca_df.PC1, pca_df.PC2)
plt.title('My PCA Graph')
plt.xlabel('PC1 - {0}%'.format(per_var[0]))
plt.ylabel('PC2 - {0}%'.format(per_var[1]))
 
for sample in pca_df.index:
    plt.annotate(sample, (pca_df.PC1.loc[sample], pca_df.PC2.loc[sample]))
 
plt.show()
 
#########################
#
# Determine which genes had the biggest influence on PC1
#
#########################
 
## get the name of the top 10 measurements (genes) that contribute
## most to pc1.
## first, get the loading scores
loading_scores = pd.Series(pca.components_[0], index=genes)
## now sort the loading scores based on their magnitude
sorted_loading_scores = loading_scores.abs().sort_values(ascending=False)
 
# get the names of the top 10 genes
top_10_genes = sorted_loading_scores[0:10].index.values
 
## print the gene names and their scores (and +/- sign)
print(loading_scores[top_10_genes])